In [1]:
import django
# from absl import app
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', "ORM.settings")
django.setup()
from nasbench import api
from nasbench101.models import NASBench101Result
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

/home/satan/.conda/envs/nas/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/satan/.conda/envs/nas/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/satan/.conda/envs/nas/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/satan/.conda/envs/nas/lib/python3.7/site-packages/tensorflow/

In [5]:
NASBENCH_TFRECORD = 'nasbench_full.tfrecord'

INPUT = 'input'
OUTPUT = 'output'
CONV1X1 = 'conv1x1-bn-relu'
CONV3X3 = 'conv3x3-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
NASBENCH_TFRECORD = '/home/satan/Downloads/nasbench_full.tfrecord'
nasbench = api.NASBench(NASBENCH_TFRECORD)

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 127 seconds


In [6]:
print(len(nasbench.fixed_statistics), len(nasbench.computed_statistics))
ans = []
for index, i in enumerate(nasbench.fixed_statistics):
    if index % 1024 == 0:
        print(index)
    try:
        db = NASBench101Result.objects.get(index=i)
        fix = nasbench.fixed_statistics[i]
        try:
            fix["module_adjacency"] = fix["module_adjacency"].tolist()
        except Exception:
            pass
        for j in fix:
            assert fix[j] == db.phenotype[j]
        # print(db.result.keys())
        # print(nasbench.computed_statistics[i])
        # assert db.result == nasbench.computed_statistics[i]
        assert db.epoch4 == nasbench.computed_statistics[i][4]
        assert db.epoch12==nasbench.computed_statistics[i][12]
        assert db.epoch36==nasbench.computed_statistics[i][36]
        assert db.epoch108==nasbench.computed_statistics[i][108]
    except Exception as e:
        ans.append(i)
        print(str(e))
    continue

423624 423624
0
1024
2048
3072
4096
5120
6144
7168
8192
9216
10240
11264
12288
13312
14336
15360
16384
17408
18432
19456
20480
21504
22528
23552
24576
25600
26624
27648
28672
29696
30720
31744
32768
33792
34816
35840
36864
37888
38912
39936
40960
41984
43008
44032
45056
46080
47104
48128
49152
50176
51200
52224
53248
54272
55296
56320
57344
58368
59392
60416
61440
62464
63488
64512
65536
66560
67584
68608
69632
70656
71680
72704
73728
74752
75776
76800
77824
78848
79872
80896
81920
82944
83968
84992
86016
87040
88064
89088
90112
91136
92160
93184
94208
95232
96256
97280
98304
99328
100352
101376
102400
103424
104448
105472
106496
107520
108544
109568
110592
111616
112640
113664
114688
115712
116736
117760
118784
119808
120832
121856
122880
123904
124928
125952
126976
128000
129024
130048
131072
132096
133120
134144
135168
136192
137216
138240
139264
140288
141312
142336
143360
144384
145408
146432
147456
148480
149504
150528
151552
152576
153600
154624
155648
156672
157696
158720
15974

In [18]:
# print(ans)
for i in ans:
    fix = nasbench.fixed_statistics[i]
    try:
        fix["module_adjacency"] = fix["module_adjacency"].tolist()
    except Exception:
        pass
    NASBench101Result.objects.create(
                index=i,
                phenotype=fix,
                result=nasbench.computed_statistics[i],
                epoch4=nasbench.computed_statistics[i][4],
                epoch12=nasbench.computed_statistics[i][12],
                epoch36=nasbench.computed_statistics[i][36],
                epoch108=nasbench.computed_statistics[i][108]
            )

['ffdb011050efcf8516d8a4a7c67e3fc5', 'ffdb08bd06f95a4bcf3599053da836d1', 'ffdb313f5b19f7c07544148119bf3e0a', 'ffdb3eb62bab1a91c824b11e48e5dbd4', 'ffdb5b2f5401fa39cece8aa042f7cd58', 'ffdb696f2b1637f641c1cb9da602b677', 'ffdb6a060e2e37c26a607786e3522a27', 'ffdb6c27a8d28d234c5c48f31be2a48d', 'ffdb810fbf586a71895ac82d24f06ca7', 'ffdb9fb773b663b0f0f81c35de9cd285', 'ffdbb6a12c59b7915f917ede469fcd35', 'ffdbda34fd5a24199501849b2b69ef35', 'ffdbf5a8faf773e523574730eba9d0f9', 'ffdbf6b7b3f07ac4272ae333115fca9e', 'ffdc601fb533862aac9c3c4794d4abc9', 'ffdc6dde752963b65d6ea5d4884a7a44', 'ffdcb578defe5a8e2f5325d6a2056c84', 'ffdd4c47a6323fb33705f2a795f7aba6', 'ffdd5a7baa69c59a7b99abca8b15742c', 'ffdda80261482f9085365a29cafbf74b', 'ffde045fb55169a888c3dfbe3ceceb9e', 'ffde14c4d29531c5a08a27d7e607086f', 'ffde44411eb96ca0ed4b9500c2c526a8', 'ffde4f9366fd6858d0bd383865128973', 'ffde87d5766ac8f9df08a611aedc4457', 'ffdedf16ff8ce10e3d9a9149ca285dd2', 'ffdeeaec6ff94b8ac1acc063a28198f8', 'ffdf34b1e4435de366de4eff55

In [4]:
# assert NASBench101Result.objects.all().count() == 423624
print(NASBench101Result.objects.get(pk=1).phenotype)
# print(NASBench101Result.objects.get(index=ans[0]).phenotype)
ans = ['ffdb011050efcf8516d8a4a7c67e3fc5', 'ffdb08bd06f95a4bcf3599053da836d1', 'ffdb313f5b19f7c07544148119bf3e0a', 'ffdb3eb62bab1a91c824b11e48e5dbd4', 'ffdb5b2f5401fa39cece8aa042f7cd58', 'ffdb696f2b1637f641c1cb9da602b677', 'ffdb6a060e2e37c26a607786e3522a27', 'ffdb6c27a8d28d234c5c48f31be2a48d', 'ffdb810fbf586a71895ac82d24f06ca7', 'ffdb9fb773b663b0f0f81c35de9cd285', 'ffdbb6a12c59b7915f917ede469fcd35', 'ffdbda34fd5a24199501849b2b69ef35', 'ffdbf5a8faf773e523574730eba9d0f9', 'ffdbf6b7b3f07ac4272ae333115fca9e', 'ffdc601fb533862aac9c3c4794d4abc9', 'ffdc6dde752963b65d6ea5d4884a7a44', 'ffdcb578defe5a8e2f5325d6a2056c84', 'ffdd4c47a6323fb33705f2a795f7aba6', 'ffdd5a7baa69c59a7b99abca8b15742c', 'ffdda80261482f9085365a29cafbf74b', 'ffde045fb55169a888c3dfbe3ceceb9e', 'ffde14c4d29531c5a08a27d7e607086f', 'ffde44411eb96ca0ed4b9500c2c526a8', 'ffde4f9366fd6858d0bd383865128973', 'ffde87d5766ac8f9df08a611aedc4457', 'ffdedf16ff8ce10e3d9a9149ca285dd2', 'ffdeeaec6ff94b8ac1acc063a28198f8', 'ffdf34b1e4435de366de4eff55c9b5e4', 'ffdf5377600d0db9a3c8bfaf7015cab8', 'ffdf56be5b9e2108b07de68c2e39e40c', 'ffdf69cc6ecd464e4e6dc9b75d33592f', 'ffdf7a4af0fd932e7a73a4ea832fe68d', 'ffdfad1fc25668a0fd4e9411f4ab6330', 'ffe01b9f963912774a31513ee70c7745', 'ffe027773329250fc876e893e4b011b9', 'ffe069216ae476818b016b19b913bda7', 'ffe0720818032d227964a501505aabcb', 'ffe08ef40f99d27158d75d162542a396', 'ffe0e2593f0542d9e987582c192c4c1b', 'ffe0e8727e7707e3dd523cfb9d933bd7', 'ffe0fcfea7bf26a33863fd6cd7c1ed93', 'ffe1150a7dec93d2beaabefef4c13ccd', 'ffe1361c5dfe76afd8609a6fab5d7a4b', 'ffe13f84fddb1b0387fc3b5246e2f549', 'ffe17c6135321a6c2b76e15fd287e4f0', 'ffe17e403f8ac7c9cb355d4d4b5fc31e', 'ffe19413466b442164443dece38b9940', 'ffe1c29b705156f7d2aa93b1f6e381c2', 'ffe1c5eedced50941710f30d37007846', 'ffe1ce33c5714f39b6b5d2468b7021cf', 'ffe2003d2d8e837069d9d8fd2f8ce95b', 'ffe21b3377b42cce3d793d4ccc687380', 'ffe222c1948cd5c6d5f653bf32264342', 'ffe2234ad2a7523fb8c667fba44c32f0', 'ffe229c42f4b42e1f3e880b86d4631ea', 'ffe249977879dd5a24573c41d019c5f0', 'ffe25ed1d163a9b53cb5b242ea5afd23', 'ffe27ed9c72474dfbf78cc361d6e91e6', 'ffe2a90e857b06ac437f928e23b32ef2', 'ffe2baba5ffbe80b87409595e2cca3ef', 'ffe2d500e7dba21b44993afda993ea1c', 'ffe3083003ee4330fd05bbd2ab0a303e', 'ffe32453b9ab44cb883c55ce6ca7cd22', 'ffe337ea1ee641ce10bccfec5142bebe', 'ffe3936988e3ee2c34a65d66e7f4173f', 'ffe3aa514dfa94cea6e8b2661cb27270', 'ffe3ace69a69cf12275a094485bd51c7', 'ffe3e724c26823245309d2cdb6d04cf9', 'ffe3f75ac6bc1d52e4688257b3e626c3', 'ffe40fdd958141687a6fb620ded6f270', 'ffe4ce8b82b9b3d80c820de57ccda436', 'ffe5937bee6c677d211a328555e193c7', 'ffe5c48360026719450572e6cf1264d9', 'ffe61fc70d4d19fdf66ff8568f61c62d', 'ffe65bcc79b7ecce1b02313d234de247', 'ffe66b1e059edf80781c71502cdb4d1f', 'ffe69d0f6b7dff38cc342bf11ab6f56b', 'ffe6ca1da3ef8fe1aff6d312d3a8cc16', 'ffe6dad31d843ce67011e60e94305036', 'ffe710e9d2fe4ae77cc894c9b9df2b3a', 'ffe73a76b40fbb04204aa2c872ffbef8', 'ffe74d09fd4e98aac00ea3b8327d0403', 'ffe7a7bea20be438e24127b4b45e7b6e', 'ffe7b66bd82895e47d7506b66eb1ef9a', 'ffe7b9e3e0df789481814c2568529235', 'ffe7d4bc059966004516b2e6abe6e278', 'ffe8e90c0d24544fcefefbb60586411d', 'ffe8f042cca364583f74ac5c7a06250b', 'ffe922e548fcd7637c96be51890549d5', 'ffe93a59bafad381e91192a6c8e046f6', 'ffe9855ba0a57f2ed8497d43fc368d43', 'ffe9d173491ee18b1f4766d55aef4fb3', 'ffe9e290a5aef53d952be6fd2312b3c5', 'ffea080b4c014ff933c922c32e6b0ada', 'ffea8b7d7045aeec60da1982eed068e4', 'ffeb1f2951e84f14ff707a831b8176a2', 'ffeb8878b6e8ce76764e83979acfba29', 'ffebb24cd0c7ecf7534e98156249f800', 'ffebcc73f06dd6f5a24c51b408b6d3f9', 'ffec4465488526f23fd1ac2d718596f8', 'ffec89789e2eeba5dfa45d5d84a5619e', 'ffec94f3a376ed7d3c99af25ac95e752', 'ffeca7a183f8513f39027f5db05b3d6b', 'ffecc07af6d3da5ccff87fde9de75239', 'ffecf8f4c155f0d8851de23009129d56', 'ffed3efe1d28e7c0572a2faec0f94650', 'ffed6bb9ce84f335fefb83007d4fb25e', 'ffed7c95b62f850f27df0fcbb4f8915c', 'ffee1ad0417a673346f3cbc246d03972', 'ffee276ce74bceb596384f16f4341b95', 'ffee3f6e3830f5278cc630b1725c89e0', 'ffee5d2f6dad7b55a80a34ddb827917f', 'ffee7b8ae359a6c007b8ac286d06c49d', 'ffeea59c123b213ef6c3b9eddbf37dea', 'ffeeaa98ce02f321d324beb59580e7cd', 'ffeeb300a462671fed95eb31b1d1fa00', 'ffeeb66bb93cc5b6d897515de6baf14b', 'ffef0916340ae0e99bbc9033605eca65', 'ffef4a61715204f8395186c98dd604af', 'ffefa1b0a55cd694722ed02a4e081c3f', 'ffefbf3ae2b7334de07a6dd42ae3aab7', 'ffefc5276cab8b95f675f849fb6212fe', 'fff04b6c0c965012d5a610ff02d7cff6', 'fff0bfa0c7ffa1f2487f5a549fb7f057', 'fff0c2614e6aa4419e5776284e5094ea', 'fff0c9c7d8826ddd949823a245927898', 'fff13558f61dac7bd591e815c21718de', 'fff166be744a47a2bfb4d5e5d1db25df', 'fff19bf416457ace433132a3244fa40e', 'fff1de8f266e6b72dc65736a5a4268cb', 'fff21a86930aaf590e738c82b7e20385', 'fff28e49c4d76d123bfb3e82dd5d89cb', 'fff32dc4d657c67d2cee01b055d49d8e', 'fff33010a035a942cc338846df46cb3d', 'fff3cd9318e3bc3940766e694e3e9709', 'fff3f896a3adc49ce4f882543777d35a', 'fff3f901061d22dd8e4f6c842d47a873', 'fff40a7c8bf9521eeb6a70506d3d5f00', 'fff41eeebb67ad84006960f2b65bf0db', 'fff441059c5b50a338c6073a842a0110', 'fff455fcba03465b169e8ec02dc67070', 'fff45bd98c3cfeebcad5110be0b5fdd3', 'fff4a06a7416b0b78ea7f468cff8041c', 'fff4cdadaded5f999472fde07330dd56', 'fff506b80556540762de6efef887f61b', 'fff517b5f991923825ed9537f89c982e', 'fff51af7bd88d46379bdea0ef2e55d8f', 'fff51cdc97c7da6fab6a0fe20a8750b9', 'fff5284df3dc41bf822428116ed1b5c7', 'fff593804f1a8ad925acb16ebb389299', 'fff5d27216ccff81e847a20534341c9c', 'fff5db082a0529b5a3d0ebc30beedddd', 'fff6595c951f154021498b3b03d4ac87', 'fff65ae2fbecd03e6087ed0a40191a8d', 'fff667049a437c27a164bbbecfb7066d', 'fff6ab829a2e3d72da6edcb7f39b818c', 'fff6b2eadce6c86ee8eb55c4ca072e91', 'fff6d152f9a86907a8f3390c6e41f574', 'fff6d382041b8cda749ff8ee8f582cf5', 'fff7765ec3271d9679bf66800142e7c9', 'fff7a221cc2e1c6e323d41912e7997ea', 'fff7de4761e5b4294f598a545209c988', 'fff7ed92ea5c79e00ac25be7cff5db32', 'fff8075340b6df897ad47d8c629b6f01', 'fff84f3d0aab2451019b8bc8c95ed809', 'fff8575234768b1ee9b2e6b6a0b130f3', 'fff89896ad0a2e49790f3100ead3637c', 'fff9332eb9e63756c21939786757b84b', 'fff982317f7427764ac1c0bf291466d8', 'fff991deabd2c2a0b6e55038ff7d7b65', 'fff9ec224c7fbf522e16659eb33a4b54', 'fff9eeb30f290904a7d7b9ab9616542e', 'fffa25e221f22e3e558a0202972ee35d', 'fffa79ab7967057d55c3375354ca19af', 'fffab7b1bb4b7ed995c5028e4c538a9e', 'fffab8dbb79cd4e991d9b6af198880a5', 'fffabf35bc181f14bc6551494d645252', 'fffb74c84b9471608a5362470415c003', 'fffb77eea8bb61d8b52a0a27946686bd', 'fffb9f65a9e3b0ba61db1d2aa9afb51f', 'fffbc3e91555538d3144cd66fa54b0bd', 'fffc0291a79fcc0947519d00795e520f', 'fffc2157e3cef7f3f27f6a03bb3c6a81', 'fffcc21ef138f33da597340656292201', 'fffd03cc13a06ccadb9cf1f125a46fe7', 'fffd38ba888c554e943c417e5b37f247', 'fffd576d745d9e2a4a8b1b40ffb29afd', 'fffd8e6bb1071759ecd2b7f89f6008c7', 'fffda8cf4bb37f7d88b23657bfcf3403', 'fffdb3f819837e81d603c7888aa55287', 'fffdd85562df7db98816386a494300b9', 'fffdf0b3360fa89cc35f50a7638dc879', 'fffe049482f28a7c4ff172275d17e8dd', 'fffe6bed888e31acae3c967ad2a925e1', 'fffe87371a00053dcdc72e421d8e312a', 'fffea010f2fdcda30cbac607d2871d54', 'fffea30fcc1e911a84234f202efb0bc9', 'ffff751485d42cff00fd97365339ff39', 'ffff76fafa7b0cdf545fd7feee9c8bf9']
import json
flops = json.load(open('hash_flops_all.json', 'r'))
for i in ans:
    x = NASBench101Result.objects.get(index = i)
    x.phenotype['flops'] = flops[i]
    x.save()
    print(flops[i])

{'module_adjacency': [[0, 1, 0, 0, 1, 1, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0]], 'module_operations': ['input', 'conv3x3-bn-relu', 'maxpool3x3', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'conv1x1-bn-relu', 'output'], 'trainable_parameters': 8555530, 'flops': 2527684618}
1073085962
350873610
1037953034
1510467594
845654026
1789143050
14135797770
1379133450
440848586
2527684618
861150026
1183416522
581527562
3612092426
1924048906
1433084490
1985865738
8739727370
1185163274
1924048906
347803722
1985521674
912451850
6296446986
9303959562
559695498
1851303946
520570890
732807882
440618570
1113106442
1779247114
1733044234
1096396938
3584731146
8739727370
6296446986
703925322
4444760074
1237084170
374908938
1185163274
883758282
889308490
1113106442
6592243722
6592243722
4142082058
1982769162
2589845514
873015306
380109770
6860679178
1172170762
560333194
1996105738
8739727370
857687626
4148963338

try to create the NAS-Bench-201 api from /home/satan/Downloads/NAS-Bench-201-v1_1-096897.pth


{'cifar10': {'edgegpu_latency': 5.807418537139893,
  'edgegpu_energy': 24.226614330768584,
  'raspi4_latency': 10.481976820010459,
  'edgetpu_latency': 0.9571811309997429,
  'pixel3_latency': 3.6058499999999998,
  'eyeriss_latency': 3.645620000000001,
  'eyeriss_energy': 0.6872827644999999,
  'eyeriss_arithmetic_intensity': 6.289297634665844,
  'fpga_latency': 2.57296,
  'fpga_energy': 18.01072,
  'average_hw_metric': 591005.8676719831},
 'cifar100': {'edgegpu_latency': 5.866079330444336,
  'edgegpu_energy': 24.471327606836955,
  'raspi4_latency': 10.432981719932286,
  'edgetpu_latency': 0.9669485199992778,
  'pixel3_latency': 5.04007,
  'eyeriss_latency': 3.648640000000001,
  'eyeriss_energy': 0.6874206219999999,
  'eyeriss_arithmetic_intensity': 6.267465213714637,
  'fpga_latency': 2.57296,
  'fpga_energy': 18.01072,
  'average_hw_metric': 848343.0600009117},
 'ImageNet16-120': {'edgegpu_latency': 5.828042030334473,
  'edgegpu_energy': 24.238826804161075,
  'raspi4_latency': 2.973176